In [26]:
import os
import os.path as osp
import numpy as np
import math
import cv2
import argparse

METER_SHAPE = 512
CIRCLE_CENTER = [256, 256]
CIRCLE_RADIUS = 250
PI = 3.1415926536
LINE_HEIGHT = 120
LINE_WIDTH = 1570  # 任意长度，之后可改成周长
TYPE_THRESHOLD = 30
METER_CONFIG = [{
    'scale_value': 0.4 / 40.0,
    'range': 25.0,
    'unit': "(MPa)"
}, {
    'scale_value': 1.6 / 32.0,
    'range': 1.6,
    'unit': "(MPa)"
}]

def read_process(label_maps):
    label_maps = use_erode_image(label_maps)
    # Convert the circular meter into rectangular meter
    line_images = creat_line_image(label_maps)
    cv2.imshow('line_image',line_image)
    # Convert the 2d meter into 1d meter
    scale_data, pointer_data = convert_1d_data(line_images)
    # Fliter scale data whose value is lower than the mean value
    scale_mean_filtration(scale_data)
    # Get scale_num, scales and ratio of meters
    result = get_meter_reader(scale_data, pointer_data)
    return result

# 腐蚀
def use_erode_image(meter_image, erode_kernel=3):
    kernel = np.ones((erode_kernel, erode_kernel), np.uint8)
    erode_image = cv2.erode(meter_image, kernel)
    return erode_image

# 转矩形
def creat_line_image( meter_image):
    line_image = np.zeros((LINE_HEIGHT, LINE_WIDTH), dtype=np.uint8)
    for row in range(LINE_HEIGHT):
        for col in range(LINE_WIDTH):
            theta = PI * 2 / LINE_WIDTH * (col + 1)
            rho = CIRCLE_RADIUS - row - 1
            x = int(CIRCLE_CENTER[0] + rho * math.cos(theta) + 0.5)
            y = int(CIRCLE_CENTER[1] - rho * math.sin(theta) + 0.5)
            line_image[row, col] = meter_image[x, y]
            # if col <40:  # TODO :为什么line_image不是从theta = 0开始
            #     cv2.circle(meter_image, (x,y), 1, 255, 4)
            #     cv2.circle(line_image, (row, col), 1, 155, 4)
            #     print(x, y, row, col, theta, meter_image[x, y])
    return line_image

# 转1D
def convert_1d_data( meter_image):
    scale_data = np.zeros((LINE_WIDTH), dtype=np.uint8)
    pointer_data = np.zeros((LINE_WIDTH), dtype=np.uint8)
    for col in range(LINE_WIDTH):
        for row in range(LINE_HEIGHT):
            if meter_image[row, col] == 38:
                pointer_data[col] += 1
            elif meter_image[row, col] == 75:
                scale_data[col] += 1
    return scale_data, pointer_data


def scale_mean_filtration( scale_data):
    mean_data = np.mean(scale_data)
    for col in range(LINE_WIDTH):
        if scale_data[col] < mean_data:
            scale_data[col] = 0

def get_meter_reader(scale_data, pointer_data):
    scale_flag = False
    pointer_flag = False
    one_scale_start = 0
    one_scale_end = 0
    one_pointer_start = 0
    one_pointer_end = 0
    scale_location = list()
    pointer_location = 0
    max_pointer_location = 0
    for i in range(LINE_WIDTH - 1):
        if scale_data[i] > 0 and scale_data[i + 1] > 0:
            if scale_flag == False:
                one_scale_start = i
                scale_flag = True
        if scale_flag:
            if scale_data[i] == 0 and scale_data[i + 1] == 0:
                one_scale_end = i - 1
                one_scale_location = (one_scale_start + one_scale_end) / 2
                scale_location.append(one_scale_location)
                one_scale_start = 0
                one_scale_end = 0
                scale_flag = False
        if pointer_data[i] > 0 and pointer_data[i + 1] > 0:
            if pointer_flag == False:
                one_pointer_start = i
                pointer_flag = True
        if pointer_flag:
            if pointer_data[i] == 0 and pointer_data[i + 1] == 0:
                one_pointer_end = i - 1
                # 去除杂点，选取最大指针距离的
                if (one_pointer_end - one_pointer_start) > max_pointer_location:
                    pointer_location = (one_pointer_start + one_pointer_end) / 2
                    max_pointer_location = one_pointer_end - one_pointer_start
                one_pointer_start = 0
                one_pointer_end = 0
                pointer_flag = False
                    
    scale_num = len(scale_location)
    scales = -1
    ratio = -1
    if scale_num > 0:
        for i in range(scale_num - 1):
            if scale_location[i] <= pointer_location and pointer_location < scale_location[i + 1]:
                scales = i + 1 + (pointer_location - scale_location[i]) / (scale_location[i + 1] - scale_location[i] + 1e-05) 
                break
        ratio = (pointer_location - scale_location[0]) / (scale_location[scale_num - 1] - scale_location[0] + 1e-05)
    result = {'scale_num': scale_num, 'scales': scales, 'ratio': ratio}
    return result



In [37]:
# 测试检测指针度数
import glob
import random


if __name__ == '__main__':
    src_img = cv2.imread('E:\\sc\\image_data\\meter\\meter_seg\\images\\val\\meter_36_mask.png', 0)
    # print(src_img.shape)
    # magic number
    kernelsize = 3

    src_img = cv2.resize(src_img, (METER_SHAPE, METER_SHAPE))
    erosion_img = use_erode_image(src_img, 3)
    result = read_process(erosion_img)
    print(result)

    if result['scale_num'] > TYPE_THRESHOLD:
        value = result['scales'] * METER_CONFIG[0]['scale_value']
    else:
        value = result['scales'] * METER_CONFIG[1]['scale_value']
    print("-- Meter result: {} --\n".format(value))

    cv2.imshow('src_img',erosion_img)
    # cv2.imshow('line_image',line_image)
    cv2.waitKey(0)

{'scale_num': 40, 'scales': 8.931034161712358, 'ratio': 0.20033955687317864}
-- Meter result: 0.08931034161712358 --



-1